# Explain Machine Learning Models

**ODSC Europe workshop - 18 September 2020 -  Margriet Groenendijk** 

*Code from [these examples](https://github.com/Trusted-AI/AIX360/tree/master/examples) is used and adapted.*

## More workshops

<div class="alert alert-success">
 <b>CROWDCAST - </b> 
 https://www.crowdcast.io/ibmdevelopereurope 
</div>

<div class="alert alert-info">
 <b>TWITCH - </b> 
 https://www.twitch.tv/ibmdeveloper
</div>

<div class="alert alert-danger">
 <b>MEETUP - </b> 
 https://www.meetup.com/IBM-Code-London
</div>

<div class="alert alert-warning">
 <b>CODE@THINK - </b> 
 https://www.ibm.com/uk-en/events/think-summit/codeatthink.html
</div>

## More info

<div class="alert alert-success">
 <b>DEMO - </b> 
 https://aix360.mybluemix.net
</div>

<div class="alert alert-info">
 <b>PROJECT - </b> 
 https://developer.ibm.com/open/projects/ai-explainability/
</div>

<div class="alert alert-danger">
 <b>REPO - </b> 
 https://github.com/Trusted-AI/AIX360
</div>

<div class="alert alert-warning">
 <b>DOCS - </b> 
 https://aix360.readthedocs.io/en/latest/index.html
</div>


## AIX360: Different Ways to explain

**End users/customers (trust)**
* Doctors: Why did you recommend this treatment?
* Customers: Why was my loan denied?   
* Teachers: Why was my teaching evaluated in this way?

**Gov’t/regulators (compliance, safety)**
* Prove to me that you didn't discriminate.

**Developers (quality, “debuggability”)**
* Is our system performing well?   
* How can we improve it?

<img src="https://github.com/MargrietGroenendijk/gitbooks/blob/master/files/tree1.png?raw=true" alt="Tree" style="width: 1000px;" align="left"/>

### Outline

* [Understanding model predictions with SHAP](#SHAP)
* [Understanding model predictions with LIME](#LIME)
* [Understanding models with TED](#TED)
* [Understanding data with ProtoDash](#data)
* [Credit Approval Tutorial](#credit)

<img src="http://aix360.mybluemix.net/static/images/methods-choice.gif" alt="Tree" style="width: 1000px;" align="left"/>

## Install and import packages

In [ ]:
#!pip install aif360
#!pip install aix360

In [ ]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import svm     
import time
np.random.seed(1)

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets.lime_encoder import LimeEncoder
from aif360.datasets import GermanDataset
from aif360.algorithms.preprocessing import Reweighing

from aix360.algorithms.protodash import ProtodashExplainer
from aix360.algorithms.ted.TED_Cartesian import TED_CartesianExplainer
from aix360.algorithms.shap import KernelExplainer
from aix360.datasets.cdc_dataset import CDCDataset
from aix360.datasets.ted_dataset import TEDDataset

import lime
import lime.lime_tabular

import shap

from IPython.display import Markdown, display
%matplotlib inline

<a class="anchor" id="SHAP"></a>
# Understanding model predictions with [SHAP](https://github.com/slundberg/shap)


<img src="https://raw.githubusercontent.com/slundberg/shap/master/docs/artwork/shap_header.png" alt="Tree" style="width: 800px;"/>


<a class="anchor" id="explain-single-shap"></a>
## Explain a single prediction

A simple example with a K nearest neighbors ([knn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)) classification of the IRIS dataset based on the [original SHAP tutorial](https://slundberg.github.io/shap/notebooks/Iris%20classification%20with%20scikit-learn.html).

Learn more about SHAP in [this chapter](https://christophm.github.io/interpretable-ml-book/shap.html#shap-summary-plot) in the Interpretable Machine Learning by Christoph Molnar.

The goal of SHAP is to explain the prediction of an instance x by computing the contribution of each feature to the prediction. Features with large absolute Shapley values are important. 

In [ ]:
shap.datasets.iris()

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(*shap.datasets.iris(), test_size=0.2, random_state=0)

In [ ]:
X_train.head()

In [ ]:
Y_train

In [ ]:
def print_accuracy(f):
    print("Accuracy = {0}%".format(100*np.sum(f(X_test) == Y_test)/len(Y_test)))
    time.sleep(0.5) # to let the print get out before any progress bars

shap.initjs()

n_neighbors = 5   # default=5
weights='uniform'  # 'uniform' or 'distance'
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
knn.fit(X_train, Y_train)

print_accuracy(knn.predict)

In [ ]:
# probability estimates
knn.predict_proba(X_train)

In [ ]:
shapexplainer = KernelExplainer(knn.predict_proba, X_train)

In [ ]:
# aix360 style for explaining input instances
shap_values = shapexplainer.explain_instance(X_test.iloc[0,:])

In [ ]:
shap_values

In [ ]:
X_test.iloc[0,:]

### The individual force plot

Red/blue: Features that push the prediction higher (to the right) are shown in red, and those pushing the prediction lower are in blue.

The plot is centered on the x-axis at explainer.expected_value. All SHAP values are relative to the model's expected value like a linear model's effects are relative to the intercept.

In [ ]:
shapexplainer.explainer.expected_value[0]

In [ ]:
shap_values[0]

In [ ]:
shap.force_plot(shapexplainer.explainer.expected_value[0], shap_values[0], X_test.iloc[0,:])

In [ ]:
X_test.iloc[23,:]

In [ ]:
shap_values = shapexplainer.explain_instance(X_test.iloc[23,:])
shap.force_plot(shapexplainer.explainer.expected_value[0], shap_values[0], X_test.iloc[23,:])

<a class="anchor" id="explain-all-shap"></a>
## Explain all predictions

In [ ]:
shap_values_all = shapexplainer.explain_instance(X_test)
shap_values_all

In [ ]:
shap.summary_plot(shap_values_all, X_test, plot_type="bar")

In [ ]:
# aix360 style for explaining input instances
shap_values = shapexplainer.explain_instance(X_test)
shap.force_plot(shapexplainer.explainer.expected_value[0], shap_values[0], X_test)

<a class="anchor" id="LIME"></a>
# Understanding model predictions with LIME

Local Interpretable Model-Agnostic Explanations

[LIME](https://lime-ml.readthedocs.io/en/latest/)

In [ ]:
aif360_location = !python -c "from distutils.sysconfig import get_python_lib; print(get_python_lib())"
import os
install_loc = os.path.join(aif360_location[0], "aif360/data/raw/german/")
%cd $install_loc

In [ ]:
!wget ftp://ftp.ics.uci.edu/pub/machine-learning-databases/statlog/german/german.data
!wget ftp://ftp.ics.uci.edu/pub/machine-learning-databases/statlog/german/german.doc
%cd -

In [ ]:
dataset_german = GermanDataset(protected_attribute_names=['age'],
                    privileged_classes=[lambda x: x >= 25],      
                    features_to_drop=['personal_status', 'sex']) 

dataset_german_train, dataset_german_test = dataset_german.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

In [ ]:
# scale data
scale_german = StandardScaler().fit(dataset_german_train.features)

X_train = scale_german.transform(dataset_german_train.features)
y_train = dataset_german_train.labels.ravel()
w_train = dataset_german_train.instance_weights.ravel()

X_test = scale_german.transform(dataset_german_test.features)
y_test = dataset_german_test.labels.ravel()
w_test = dataset_german_test.instance_weights.ravel()

In [ ]:
# reweigh the data
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)

# compute the weights for reweighing the dataset
RW.fit(dataset_german_train)

# transform the dataset to a new dataset based on the estimated transformation
dataset_transf_train = RW.transform(dataset_german_train)
dataset_transf_test = RW.transform(dataset_german_test)

scale_transf = StandardScaler().fit(dataset_transf_train.features)

X_train_transf = scale_transf.transform(dataset_transf_train.features)
y_train_transf = dataset_transf_train.labels.ravel()
w_train_transf = dataset_transf_train.instance_weights.ravel()

X_test_transf = scale_transf.transform(dataset_transf_test.features)
y_test_transf = dataset_transf_test.labels.ravel()
w_test_transf = dataset_transf_test.instance_weights.ravel()

In [ ]:
metric_german_train = BinaryLabelDatasetMetric(dataset_german_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)


display(Markdown("#### BIAS METRICS"))
display(Markdown("#### Original training dataset"))
print("mean_difference = %f" % metric_german_train.mean_difference())
print("disparate_impact = %f" % metric_german_train.disparate_impact())

metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)

display(Markdown("#### Reweighted training dataset"))
print("mean_difference = %f" % metric_transf_train.mean_difference())
print("disparate_impact = %f" % metric_transf_train.disparate_impact())

In [ ]:
lmod_transf = LogisticRegression()

# train the model
lmod_transf.fit(X_train_transf, y_train_transf, 
         sample_weight=dataset_transf_train.instance_weights)

# calculate predicted labels
y_train_pred_transf = lmod_transf.predict(X_train_transf)

# assign positive class index
pos_ind_transf = np.where(lmod_transf.classes_ == dataset_transf_train.favorable_label)[0][0]

# add predicted labels to predictions dataset
dataset_transf_train_pred = dataset_transf_train.copy()
dataset_transf_train_pred.labels = y_train_pred_transf

In [ ]:
limeData = LimeEncoder().fit(dataset_german_train)
s_train = limeData.transform(dataset_german_train.features)
s_test = limeData.transform(dataset_german_test.features)

scale = scale_transf
model = lmod_transf   

explainer = lime.lime_tabular.LimeTabularExplainer(s_train ,class_names=limeData.s_class_names, 
                                                   feature_names = limeData.s_feature_names,
                                                   categorical_features=limeData.s_categorical_features, 
                                                   categorical_names=limeData.s_categorical_names, 
                                                   kernel_width=3, verbose=False,discretize_continuous=True)

s_predict_fn = lambda x: model.predict_proba(scale.transform(limeData.inverse_transform(x)))

display(Markdown("#### Reweighted training dataset"))

i1 = 0
exp = explainer.explain_instance(s_test[i1], s_predict_fn, num_features=8)
exp.show_in_notebook(show_all=False)
print("        Actual label: " + str(dataset_german_test.labels[i1]))

In [ ]:
i1 = 1
exp = explainer.explain_instance(s_test[i1], s_predict_fn, num_features=8)
exp.show_in_notebook(show_all=False)
print("        Actual label: " + str(dataset_german_test.labels[i1]))

In [ ]:
i2 = 23
exp = explainer.explain_instance(s_test[i2], s_predict_fn, num_features=8)
exp.show_in_notebook(show_all=False)
print("        Actual label: " + str(dataset_german_test.labels[i2]))

<a class="anchor" id="TED"></a>
# Understanding models with TED

Most suited for use cases where matching explanations to the mental model of the explanation consumer is the highest priority; i.e., where the explanations are similar to what would be produced by a domain expert.

The **TED_CartesianExplainer** is an implementation of the algorithm in the AIES'19 paper by [Hind et al.](), that takes the Cartesian product of the label and explanation and creates a new label (YE) and uses this to train a (multiclass) classifier. 

This approach can use any classifier (passed as a parameter), as long as it complies with the fit/predict paradigm.

## Synthetic dataset

A synthetically generated [dataset](https://github.com/IBM/AIX360/blob/master/aix360/data/ted_data/Retention.csv) is used generated with this [code](https://github.com/IBM/AIX360/blob/master/aix360/data/ted_data/GenerateData.py) as part of aix360.

### Assigning labels

25 rules are created, for why a retention action is needed to reduce the chances of an employee choosing to leave a fictitious company. These rules are motivated by common scenarios, such as not getting a promotion in a while, not being paid competitively, receiving a disappointing evaluation, being a new employee in certain organizations with inherently high attrition, not having a salary that is consistent with positive evaluations, mid-career crisis, etc.   

Each of these 25 rules would result in the label "Yes"; i.e., the employee is a risk to leave the company. Because the rules capture the reason for the "Yes", we use the rule number as the explanation (E), which is required by the TED framework.

If none of the rules are satisfied, it means the employee is not a candidate for a retention action; i.e., a "No" label is assigned.  

### Dataset characteristics

10,000 fictious employees (X) are generated and the 26 (25 Yes + 1 No) rules are applied to produce Yes/No labels (Y), using these rules as explanations (E).  After applying these rules, the resulting dataset has the following characteristics:
- Yes (33.8%)
- No (66.2%)

In [ ]:
# Decompose the dataset into X, Y, E     
X, Y, E = TEDDataset().load_file('Retention.csv')
print("X's shape:", X.shape)
print("Y's shape:", Y.shape)
print("E's shape:", E.shape)
print()

# set up train/test split
X_train, X_test, Y_train, Y_test, E_train, E_test = train_test_split(X, Y, E, test_size=0.20, random_state=0)
print("X_train shape:", X_train.shape, ", X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape, ", Y_test shape:", Y_test.shape)
print("E_train shape:", E_train.shape, ", E_test shape:", E_test.shape)

In [ ]:
# Create classifier and pass to TED_CartesianExplainer
estimator = svm.SVC(kernel='linear')
# estimator = DecisionTreeClassifier()
# estimator = RandomForestClassifier()
# estimator = AdaBoostClassifier()

ted = TED_CartesianExplainer(estimator)

In [ ]:
print("Training the classifier")

ted.fit(X_train, Y_train, E_train)   # train classifier

<a class="anchor" id="ted1"></a>
## Explain a single prediction

The trained TED classifier is now ready for predictions with explanations.   We construct some raw feature vectors, created from the original dataset, and ask for a label (Y) prediction and its explanation (E).

In [ ]:
# Create an instance level example 
X1 = [[1, 2, -11, -3, -2, -2,  22, 22]]

# correct answers:  Y:-10; E:13
Y1, E1 = ted.predict_explain(X1)
print("Predicting for feature vector:")
print(" ", X1[0])
print("\t\t      Predicted \tCorrect")
print("Label(Y)\t\t " + np.array2string(Y1[0]) + "\t\t   -10")
print("Explanation (E) \t " + np.array2string(E1[0]) + "\t\t   13")
print()

In [ ]:
X2 = [[3, 1, -11, -2, -2, -2, 296, 0]]

## correct answers: Y:-11, E:25
Y2, E2 = ted.predict_explain(X2)
print("Predicting for feature vector:")
print(" ", X2[0])

print("\t\t      Predicted \tCorrect")
print("Label(Y)\t\t " + np.array2string(Y2[0]) + "\t\t   -11")
print("Explanation (E) \t " + np.array2string(E2[0]) + "\t\t   25")

### Create a more relevant human interface

TED_CaresianExplainer can produce the correct explanation for a feature vector, but simply producing "3" as an explanation is not sufficient in most uses. This section shows one way to implement the mapping of real explanations to the explanation IDs that TED requires. This is inspired by the [FICO reason codes](https://www.fico.com/en/latest-thinking/product-sheet/us-fico-score-reason-codes), which are explanations for a FICO credit score.  

In this case the explanations are text, but the same idea can be used to map explanation IDs to other formats, such as a file name containing an audio or video explanation.

In [ ]:
Label_Strings =["IS", "Approved for"]
def labelToString(label) :
    if label == -10 :
        return "IS"
    else :
        return "IS NOT"

Explanation_Strings = [
    "Seeking Higher Salary in Org 1",
    "Promotion Lag, Org 1, Position 1",
    "Promotion Lag, Org 1, Position 2",
    "Promotion Lag, Org 1, Position 3",
    "Promotion Lag, Org 2, Position 1",
    "Promotion Lag, Org 2, Position 2",
    "Promotion Lag, Org 2, Position 3",
    "Promotion Lag, Org 3, Position 1",
    "Promotion Lag, Org 3, Position 2",
    "Promotion Lag, Org 3, Position 3",
    "New employee, Org 1, Position 1",
    "New employee, Org 1, Position 2",
    "New employee, Org 1, Position 3",
    "New employee, Org 2, Position 1",
    "New employee, Org 2, Position 2",
    "Disappointing evaluation, Org 1",
    "Disappointing evaluation, Org 2",
    "Compensation does not match evaluations, Med rating",
    "Compensation does not match evaluations, High rating",
    "Compensation does not match evaluations, Org 1, Med rating",
    "Compensation does not match evaluations, Org 2, Med rating",
    "Compensation does not match evaluations, Org 1, High rating",
    "Compensation does not match evaluations, Org 2, High rating",
    "Mid-career crisis, Org 1",
    "Mid-career crisis, Org 2",
    "Did not match any retention risk rules"]


print("Employee #1 " + labelToString(Y1[0]) + " a retention risk with explanation: " + Explanation_Strings[E1[0]])
print()
print("Employee #2 " + labelToString(Y2[0]) + " a retention risk with explanation: " + Explanation_Strings[E2[0]])

<a class="anchor" id="ted2"></a>
## Overall model accuracy

How well does TED_Cartesian do in predicting all test labels (Y) and explanations (E)?

The "score" method of TED_Cartesian calculates this. The accuracy of predicting the combined YE labels could be of interest to researchers who want to better understand the inner workings of TED_Cartesian.


In [ ]:
YE_accuracy, Y_accuracy, E_accuracy = ted.score(X_test, Y_test, E_test)    # evaluate the classifier
print("Evaluating accuracy of TED-enhanced classifier on test data")
print(' Accuracy of predicting Y labels: %.2f%%' % (100*Y_accuracy))
print(' Accuracy of predicting explanations: %.2f%%' % (100*E_accuracy))
print(' Accuracy of predicting Y + explanations: %.2f%%' % (100*YE_accuracy))

* It is easy to use the TED_CartesianExplainer if you have a training dataset that contains explanations. The framework is general in that it can use any classification technique that follows the fit/predict paradigm, so that if you already have a favorite algorithm, you can use it with the TED framework.
* The main advantage of this algorithm is that the quality of the explanations produced are exactly the same quality as those that the algorithm is trained on.  Thus, if you teach (train) the system well with good training data and good explanations, you will get good explanations out in a language you should understand.
* The downside of this approach is that someone needs to create explanations. This should be straightforward when a domain expert is creating the initial training data: if they decide a loan should be rejected, they should know why, and if they do not, it may not be a good decision.
* However, this may be more of a challenge when a training dataset already exists without explanations and now someone needs to create the explanations.  The original person who did the labeling of decisions may no longer be available, so the explanations for the decisions may not be known.  In this case, we argue, the system is in a dangerous state.  Training data exists that no one understands why it is labeled in a certain way.   Asking the model to explain one of its predictions when no person can explain an instance in the training data does not seem consistent.
* Dealing with this situation is one of the open research problems that comes from the TED approach.

<a class="anchor" id="data"></a>
# Understanding data

Health and Lifestyle Survey 

The [NHANES CDC questionnaire datasets](https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Questionnaire&CycleBeginYear=2013) are surveys conducted by the organization involving thousands of civilians about their daily lives. There are 44 questionnaires that collect data about income, occupation, health, early childhood and many other behavioral and lifestyle aspects of individuals living in the US.

[Protodash: Fast Interpretable Prototype Selection](https://arxiv.org/abs/1707.01212)

* The method takes as input a datapoint (or group of datapoints) to be explained with respect to points in a training set belonging to the same feature space
* The method then tries to minimize the maximum mean discrepancy (MMD metric) between the datapoints we want to explain and a prespecified number of instances from the training set that it will select. It will try to select training instances that have the same distribution as the datapoints we want to explain
* The method has quality guarantees with it, returning importance weights for the chosen prototypical training instances indicative of how similar/representative they are

## Why Protodash?

Prototypes represent the different segments in a dataset

* For example you might find that there are three categories of people: i) those that are high earners, ii) those that are middle class and iii) those that don't earn much or are unemployed and receive unemployment benefits
* Protodash will find these segments by pointing to specific individuals that lie in these categories
* From the objective function value of Protodash you are also able to say that three segments is the right number here as adding one more segment may not improve the objective value by much
* The 'ProtodashExplainer' provides exemplar-based explanations for summarizing datasets as well as explaining predictions.

## Load data

This dataset is baked into aix360, which makes it easy to download:

In [ ]:
nhanes = CDCDataset()
nhanes_files = nhanes.get_csv_file_names()
(nhanesinfo, _, _) = nhanes._cdc_files_info()

In [ ]:
nhanes_files

Each column in the dataset corresponds to a question and each row are the answers given by a respondent to those questions. Both column names and answers by respondents are encoded. For example, 'SEQN' denotes the sequence number assigned to a respondent and 'IND235' corresponds to a question about monthly family income. In most cases a value of 1 implies "Yes" to the question, while a value of 2 implies "No." More details about the income questionaire and how questions and answers are encoded is [here](https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/INQ_H.htm).

In [ ]:
# replace encoded column names by the associated question text. 
df_inc = nhanes.get_csv_file('INQ_H.csv')
df_inc.columns[0]
dict_inc = {
'SEQN': 'Respondent sequence number', 
'INQ020': 'Income from wages/salaries',
'INQ012': 'Income from self employment',
'INQ030':'Income from Social Security or RR',
'INQ060':  'Income from other disability pension', 
'INQ080':  'Income from retirement/survivor pension',
'INQ090':  'Income from Supplemental Security Income',
'INQ132':  'Income from state/county cash assistance', 
'INQ140':  'Income from interest/dividends or rental', 
'INQ150':  'Income from other sources',
'IND235':  'Monthly family income',
'INDFMMPI':  'Family monthly poverty level index', 
'INDFMMPC':  'Family monthly poverty level category',
'INQ244':  'Family has savings more than $5000',
'IND247':  'Total savings/cash assets for the family'
}
qlist = []
for i in range(len(df_inc.columns)):
    qlist.append(dict_inc[df_inc.columns[i]])
df_inc.columns = qlist
print("Answers given by some respondents to the income questionnaire:")
df_inc.head(5).transpose()

In [ ]:
print("Number of respondents to Income questionnaire:", df_inc.shape[0])
print("Distribution of answers to \'monthly family income\' and \'Family savings\' questions:")

fig, axes = plt.subplots(1, 2, figsize=(10,5))
fig.subplots_adjust(wspace=0.5)
hist1 = df_inc['Monthly family income'].value_counts().plot(kind='bar', ax=axes[0])
hist2 = df_inc['Family has savings more than $5000'].value_counts().plot(kind='bar', ax=axes[1])
plt.show()

* The majority of individuals responded with a "12" for the question related to monthly family income, which means their income is above USD 8400 as explained [here](https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/INQ_H.htm#IND235)
* To the question of whether the family has savings of more than USD 5000, the majority of individuals responded with a "2", which means "No"

## Summarize Income Questionnaire using Prototypes

Consider a social scientist who would like to quickly obtain a summary report of this dataset in terms of types of people that span this dataset. Is it possible to summarize this dataset by looking at answers given by a few representative/prototypical respondents?

The Protodash algorithm can be used to obtain a few prototypical respondents (about 10 in this example) that span the diverse set of individuals answering the income questionnaire making it easy to summarize the dataset.

In [ ]:
# convert pandas dataframe to numpy
data = df_inc.to_numpy()

#sort the rows by sequence numbers in 1st column 
idx = np.argsort(data[:, 0])  
data = data[idx, :]

# replace nan's (missing values) with 0's
original = data
original[np.isnan(original)] = 0

# delete 1st column (sequence numbers)
original = original[:, 1:]

print(original.shape)
original

In [ ]:
# one hot encode all features as they are categorical
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(original)

In [ ]:
print(onehot_encoded.shape)
onehot_encoded

Input:

* Dataset to select prototypical explanations from: onehot_encoded
* Dataset you want to explain: also onehot_encoded
* Number of prototypes m

Output:

* Indices of the selected prototypes S
* Importance weights associated with the selected prototypes W

In [ ]:
explainer = ProtodashExplainer()

(W, S, _) = explainer.explain(onehot_encoded, onehot_encoded, m=10) 

# sort the order of prototypes in set S
idx = np.argsort(S)
S = S[idx]
W = W[idx]

In [ ]:
S

In [ ]:
W

In [ ]:
sum(W)

In [ ]:
# Display the prototypes along with their computed weights
inc_prototypes = df_inc.iloc[S, :].copy()
# Compute normalized importance weights for prototypes
inc_prototypes["Weights of Prototypes"] = np.around(W/np.sum(W), 2) 
inc_prototypes.transpose()

The 10 people shown above (i.e. 10 prototypes) are representative of the income questionnaire according to Protodash.

* In the distribution plot for family finance related questions, there roughly were five times as many people not having savings in excess of USD 5000 compared with others. The prototypes also have a similar spread, which is reassuring
* For monthly family income, there is a more even spread over the more commonly occurring categories. This is a kind of spot check to see if our prototypes actually match the distribution of values in the dataset.
* Most people are employed (3rd question) and work for an organization earning through salary/wages (1st two questions)
* Most of them are also young (5th question) and fit to work (4th question)
* They don't seem to have much savings (last question)

## Find Questionnaires that are most representative of Income

How do the remaining 39 questionnaires represent or relate to income? The below will give an idea of which lifestyle factors are likely to affect income the most from prototypes for each of the questionnaires.

### Compute prototypes for all questionaires

This step uses Protodash explainer to compute 10 prototypes for each of the questionaires and saves these for further evaluation. (This takes a little while)

In [ ]:
# Iterate through all questionnaire datasets and find 10 prototypes for each.

prototypes = {}

for i in range(len(nhanes_files)):
    
    f = nhanes_files[i]
    
    print("processing ", f)
    
    # read data to pandas dataframe
    df = nhanes.get_csv_file(f)
    
    # convert data to numpy
    data = df.to_numpy()

    #sort the rows by sequence numbers in 1st column 
    idx = np.argsort(data[:, 0])
    data = data[idx, :]

    # replace nan's with 0's.
    original = data
    original[np.isnan(original)] = 0

    # delete 1st column (contains sequence numbers)
    original = original[:, 1:]

    # one hot encode all features as they are categorical
    onehot_encoder = OneHotEncoder(sparse=False)
    onehot_encoded = onehot_encoder.fit_transform(original)

    explainer = ProtodashExplainer()

    # call Protodash explainer
    # S contains indices of the selected prototypes
    # W contains importance weights associated with the selected prototypes 

    (W, S, _) = explainer.explain(onehot_encoded, onehot_encoded, m=10) 

    prototypes[f]={}
    prototypes[f]['W']= W
    prototypes[f]['S']= S
    prototypes[f]['data'] = data
    prototypes[f]['original'] = original

## Evaluate the set of prototypical respondents from various questionaires using their income questionaire.

How well do the prototypes of each questionaire represent the Income questionnaire based on the objective function that Protodash uses?

By ranking the different questionnaires with their objective function values in ascending order. The higher a questionaire appears in the list, the better its prototypes represent the income questionaire. The values on the right indicate our objective value where lower value is better.

In [ ]:
#load income dataset INQ_H and its prototypes
X = prototypes['INQ_H.csv']['original']
Xdata = prototypes['INQ_H.csv']['data']

# Iterate through all questionnaires and evaluate how well their prototypes represent the income dataset. 
objs = []
for i in range(len(nhanes_files)):
        
    #load a dataset, its prototypes & weights

    f = nhanes_files[i]
    
    Ydata = prototypes[f]['data']
    S = prototypes[f]['S']
    W = prototypes[f]['W']
    
    
    # sort the order of prototypes in set S
    idx = np.argsort(S)
    S = S[idx]
    W = W[idx]
    
    # access corresponding prototypes in X. 
    XS = X[np.isin(Xdata[:, 0], Ydata[S, 0]), :]
    
    #print(Ydata[S, 0])
    #print(Xdata[np.isin(Xdata[:, 0], Ydata[S, 0]), 0])   
    
    temp = np.dot(XS, np.transpose(X))    
    u = np.sum(temp, axis=1)/temp.shape[1]
    
    K = np.dot(XS, XS.T)
    
    # evaluate prototypes on income based on our objective function with dot product as similarity measure
    obj = 0.5 * np.dot(np.dot(W.T, K), W) - np.dot(W.T, u)
    objs.append(obj)    
    

# sort the objectives (ascending order) 
index = np.argsort(np.array(objs))

# load the results in a dataframe to print
evalresult = []
for i in range(0,len(index)):    
    evalresult.append([ nhanesinfo[index[i]], objs[index[i]] ])
    
    
df_evalresult = pd.DataFrame.from_records(evalresult)
df_evalresult.columns = ['Questionaire', 'Prototypes representative of Income']
df_evalresult

### Insight from Protodash

* Early childhood represents income the most. The early childhood questionnaire has information about the environment that the child was born and raised in
* This is consistent with a long term study that talks about significant decrease in social mobility in recent times, stressing the fact that your childhood impacts how monetarily successful you are likely to be
* Protodash was able to uncover this relationship with access to just these survey questionnaires

<a class="anchor" id="credit"></a>
# Credit Approval Tutorial

Explore different kinds of explanations suited to different users in the context of a credit approval process enabled by machine learning

* Data from the [FICO Explainable Machine Learning Challenge](https://community.fico.com/s/explainable-machine-learning-challenge) 
* For the data scientist two directly interpretable rule-based models that provide global understanding of their behavior:
    * Boolean Rule Column Generation (BRCG, class `BooleanRuleCG`) - very simple OR-of-ANDs classification rules
    * Logistic Rule Regression (LogRR, class `LogisticRuleRegression`) - weighted combinations of rules that are more accurate and still interpretable
* For the loan officer examples, specifically _prototypes_ or representatives in the training data that are similar to a given loan applicant and receive the same class label:
    * ProtoDash (class `ProtodashExplainer`)
* For the bank customer:
    * Contrastive Explanations Method (CEM, class `CEMExplainer`) explaining the predictions of black box models to end users. 

1. [Introduction to FICO HELOC Dataset](#fico-intro)
2. [Data Scientist: Boolean Rules and Logistic Rule Regression models](#fico-rule-based-models)
3. [Loan Officer: Similar samples as explanations for predictions based on HELOC Dataset](#prototypes)
4. [Customer: Contrastive Explanations for predictions based on HELOC Dataset](#contrastive)

<a name="fico-intro"></a>
## 1. Introduction to FICO HELOC Dataset

*  Anonymized information about home equity line of credit (HELOC) applications made by real homeowners
* A HELOC is a line of credit typically offered by a US bank as a percentage of home equity (the difference between the current market value of a home and the outstanding balance of all liens, e.g. mortgages)
* The customers in this dataset have requested a credit line in the range of USD 5,000 - 150,000 
* Predict whether they will make timely payments over a two year period
* The prediction can then be used to decide whether the homeowner qualifies for a line of credit and, if so, how much credit should be extended 

<div class="alert alert-success">
 <b>DOWNLOAD THE DATA</b> <br/> 
     <b>https://community.fico.com/s/explainable-machine-learning-challenge?tabset-3158a=2<br/> 
</div>

* The target variable to predict is a binary variable called RiskPerformance
* The value “Bad” indicates that an applicant was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened
* The value “Good” indicates that they have made their payments without ever being more than 90 days overdue 
* The relationship between a predictor variable and the target is indicated in the last column of the table. 

|Field | Meaning |Monotonicity Constraint (with respect to probability of bad = 1)|
|------|---------|----------------------------------------------------------------|
|ExternalRiskEstimate |	Consolidated version of risk markers |Monotonically Decreasing| 
|MSinceOldestTradeOpen	| Months Since Oldest Trade Open | Monotonically Decreasing|
|MSinceMostRecentTradeOpen | Months Since Most Recent Trade Open |Monotonically Decreasing
|AverageMInFile	| Average Months in File |Monotonically Decreasing|
|NumSatisfactoryTrades |	Number Satisfactory Trades |Monotonically Decreasing|
|NumTrades60Ever2DerogPubRec |	Number Trades 60+ Ever |Monotonically Decreasing|
|NumTrades90Ever2DerogPubRec | Number Trades 90+ Ever |Monotonically Decreasing| 
|PercentTradesNeverDelq	| Percent Trades Never Delinquent|Monotonically Decreasing|
|MSinceMostRecentDelq	| Months Since Most Recent Delinquency|Monotonically Decreasing|
|MaxDelq2PublicRecLast12M |	Max Delq/Public Records Last 12 Months. See tab "MaxDelq" for each category|Values 0-7 are monotonically decreasing|
|MaxDelqEver |	Max Delinquency Ever. See tab "MaxDelq" for each category|Values 2-8 are monotonically decreasing|
|NumTotalTrades	| Number of Total Trades (total number of credit accounts)|No constraint|
|NumTradesOpeninLast12M	| Number of Trades Open in Last 12 Months|Monotonically Increasing| 
|PercentInstallTrades	| Percent Installment Trades|No constraint|
|MSinceMostRecentInqexcl7days |	Months Since Most Recent Inq excl 7days|Monotonically Decreasing| 
|NumInqLast6M	| Number of Inq Last 6 Months|Monotonically Increasing|
|NumInqLast6Mexcl7days	| Number of Inq Last 6 Months excl 7days. Excluding the last 7 days removes inquiries that are likely due to price comparision shopping. |Monotonically Increasing|
|NetFractionRevolvingBurden	| Net Fraction Revolving Burden. This is revolving balance divided by credit limit |Monotonically Increasing|
|NetFractionInstallBurden	| Net Fraction Installment Burden. This is installment balance divided by original loan amount |Monotonically Increasing| 
|NumRevolvingTradesWBalance	| Number Revolving Trades with Balance |No constraint|
|NumInstallTradesWBalance	| Number Installment Trades with Balance |No constraint|
|NumBank2NatlTradesWHighUtilization	| Number Bank/Natl Trades w high utilization ratio |Monotonically Increasing|
|PercentTradesWBalance	| Percent Trades with Balance |No constraint
|RiskPerformance	| Paid as negotiated flag (12-36 Months). String of Good and Bad | Target |

#### Storing HELOC dataset to run this notebook
Copy the downloaded file under site-packages of your virtual environment `path-to-your-virtual-env/lib/python3.6/site-packages/aix360/data/heloc_data/heloc_dataset.csv` For example `/opt/anaconda3/envs/aix360/lib/python3.6/site-packages`


#### Running the notebook in Watson Studio
    
An extra step is needed
* Upload the file to the project in the menu on the right side of the notebook
* Create a new **Access token** on the **Settings** page of the project - select **Editor** (quickest to do this, is in a new browser tab)   
* Go back to the notebook and click on 3 dots in the top right above the notebook to insert the **Project Token**
* This adds a cell to the top of the notebook - run this cell, and then scroll back down to continue with the rest of the notebook
* The below cell will now copy the dat file to the correct location to run the rest of the notebook (when running this notebook locally, you can ignore the error)    
    

In [ ]:
# when notebook is run on Watson Studio
def download_file_to_local(project_filename, local_file_destination=None, project=None):
    """
    Uses project-lib to get a bytearray and then downloads this file to local.
    Requires a valid `project` object.
    
    Args:
        project_filename str: the filename to be passed to get_file
        local_file_destination: the filename for the local file if different
        
    Returns:
        0 if everything worked
    """
    
    project = project
    
    # get the file
    print("Attempting to get file {}".format(project_filename))
    _bytes = project.get_file(project_filename).read()
    
    # check for new file name, download the file
    print("Downloading...")
    if local_file_destination==None: local_file_destination = project_filename
    
    with open(local_file_destination, 'wb') as f: 
        f.write(bytearray(_bytes))
        print("Completed writing to {}".format(local_file_destination))
        
    return 0

aix360_location = !python -c "from distutils.sysconfig import get_python_lib; print(get_python_lib())"
install_loc = os.path.join(aix360_location[0], "aix360/data/heloc_data/heloc_dataset.csv")
download_file_to_local('heloc_dataset.csv', local_file_destination=install_loc,project=project)

<a name="rule-based-models"></a>
## 2. Data scientist: Boolean Rule and Logistic Rule Regression models

* A data scientist would ideally like to understand the behavior of the model as a whole, not just in specific instances (e.g. specific loan applicants)
* Especially in regulated industries such as banking where higher standards of explainability may be required. 
* Directly interpretable models can provide such global understanding because they have a sufficiently simple form for their workings to be transparent
    * [Boolean rule (BR)](#BRCG) produced by the Boolean Rule Column Generation (BRCG) that yields a very simple set of rules that has reasonable accuracy
    * [logistic rule regression (LogRR)](#LogRR) is a generalized linear rule model (GLRM) that achieves higher accuracy, higher even than some uninterpretable models, while retaining the form of a linear model

### 2.1. Load and process data for BRCG and LogRR

Use the `HELOCDataset` class in AIX360 to load the FICO HELOC data as a DataFrame
* `custom_preprocessing=nan_preprocessing` converts special values in the data (coded as negative integers) to `np.nan`, which can be handled properly by BRCG and LogRR, as opposed to replacing them with zeros or mean values
* The data is split into training and test sets using a fixed random seed



In [ ]:
# Load FICO HELOC data with special values converted to np.nan
from aix360.datasets.heloc_dataset import HELOCDataset, nan_preprocessing
data = HELOCDataset(custom_preprocessing=nan_preprocessing).data()
# Separate target variable
y = data.pop('RiskPerformance')

# Split data into training and test sets using fixed random seed
from sklearn.model_selection import train_test_split
dfTrain, dfTest, yTrain, yTest = train_test_split(data, y, random_state=0, stratify=y)
dfTrain.head().transpose()

BRCG and LogRR require non-binary features to be binarized using the provided `FeatureBinarizer` class
* use the default of nine quantile thresholds (i.e. 10 bins) to binarize ordinal (including continuous-valued) features, include all negations (e.g. '>' comparisons as well as '<=')
* return standardized versions of the original unbinarized ordinal features, which are used by LogRR but not BRCG. 

In [ ]:
# Binarize data and also return standardized ordinal features
from aix360.algorithms.rbm import FeatureBinarizer
fb = FeatureBinarizer(negations=True, returnOrd=True)
dfTrain, dfTrainStd = fb.fit_transform(dfTrain)
dfTest, dfTestStd = fb.transform(dfTest)
dfTrain['ExternalRiskEstimate'].head()

<a name="BRCG"></a>
### 2.2. Run Boolean Rule Column Generation (BRCG)

* Produces a very simple OR-of-ANDs rule (disjunctive normal form, DNF) or alternatively an AND-of-ORs rule (conjunctive normal form, CNF) to predict whether an applicant will repay the loan on time (Y = 1)
* For a binary classification problem a DNF rule is equivalent to a *rule set*, where AND clauses in the DNF correspond to individual rules in the rule set
* BRCG is distinguished by its use of the optimization technique of column generation to search the space of possible clauses, which is exponential in size. 
* Learn more about column generation from this [NeurIPS paper](http://papers.nips.cc/paper/7716-boolean-decision-rules-via-column-generation). 

* For this dataset a CNF rule for Y = 1 (i.e. a DNF for Y = 0, enabled by setting `CNF=True`) is slightly better than a DNF rule for Y = 1
* The model complexity parameters `lambda0` and `lambda1` penalize the number of clauses in the rule and the number of conditions in each clause
* The default values of 1e-3 for `lambda0` and `lambda1` are used (decreasing them did not increase accuracy here) and other parameters at their defaults as well
* The model is then trained, evaluated, and printed

In [ ]:
# Instantiate BRCG with small complexity penalty and large beam search width
from aix360.algorithms.rbm import BooleanRuleCG
br = BooleanRuleCG(lambda0=1e-3, lambda1=1e-3, CNF=True)

# Train, print, and evaluate model
br.fit(dfTrain, yTrain)
from sklearn.metrics import accuracy_score
print('Training accuracy:', accuracy_score(yTrain, br.predict(dfTrain)))
print('Test accuracy:', accuracy_score(yTest, br.predict(dfTest)))
print('Predict Y=0 if ANY of the following rules are satisfied, otherwise Y=1:')
print(br.explain()['rules'])

* The returned DNF rule for Y = 0 is very simple with only two clauses, each involving the same two features
* This rule can already achieve 69.7% accuracy
* `ExternalRiskEstimate` is a consolidated version of some risk markers (higher is better)
* `NumSatisfactoryTrades` is the number of satisfactory credit accounts
* It makes sense that for applicants with more than 17 satisfactory accounts, the ExternalRiskEstimate threshold dividing good (Y = 1) and bad (Y = 0) credit risk is slightly lower (more lenient) than for applicants with fewer satisfactory accounts.

<a name="LogRR"></a>
### 2.3. Run Logistic Rule Regression (LogRR)

* A LogRR model can improve accuracy at the cost of a more complex but still interpretable model
* LogRR fits a logistic regression model using rule-based features, where column generation is again used to generate promising candidates from the space of all possible rules
* Unbinarized ordinal features (`useOrd=True`) are included in addition to rules. 
* Similar to BRCG, the complexity parameters `lambda0`, `lambda1` penalize the number of rules included in the model and the number of conditions in each rule. 
* The values for `lambda0`, `lambda1` below strike a good balance between accuracy and model complexity, see [here](http://proceedings.mlr.press/v97/wei19a.html).

In [ ]:
# Instantiate LRR with good complexity penalties and numerical features
from aix360.algorithms.rbm import LogisticRuleRegression
lrr = LogisticRuleRegression(lambda0=0.005, lambda1=0.001, useOrd=True)

# Train, print, and evaluate model
lrr.fit(dfTrain, yTrain, dfTrainStd)
print('Training accuracy:', accuracy_score(yTrain, lrr.predict(dfTrain, dfTrainStd)))
print('Test accuracy:', accuracy_score(yTest, lrr.predict(dfTest, dfTestStd)))
print('Probability of Y=1 is predicted as logistic(z) = 1 / (1 + exp(-z))')
print('where z is a linear combination of the following rules/numerical features:')
lrr.explain()

The test accuracy of LogRR is significantly better than that of BRCG and even better than the neural network in the [Loan Officer](#c2) and [Customer](#contrastive) sections. The LogRR model remains directly interpretable as it is a logistic regression model that uses the 36 rule-based and ordinal features shown above (in addition to an intercept term). Rules are distinguished by having one or more conditions on feature values (e.g. AverageMInFile <= 52.0) while ordinal features are marked by just the feature name without conditions (e.g. ExternalRiskEstimate). Being a linear model, feature importance is naturally given by the model coefficients and thus the list is sorted in order of decreasing coefficient magnitude. The list can be truncated if the user wishes to display fewer features.

Since the rules in this LogRR model happen to all be single conditions on individual features, the model contains no interactions between features. It is therefore a kind of [generalized additive model (GAM)](https://en.wikipedia.org/wiki/Generalized_additive_model), i.e. a sum of functions of individual features, where these functions are themselves sums of step function components from rules and linear components from unbinarized ordinal features. Thus a better way to visualize the model is by plotting the univariate functions that make up the GAM, as we do next.

<a name="visualize"></a>
### 2.4. Visualize LogRR model as a Generalized Additive Model (GAM)
We use the `visualize()` method of `LogisticRuleRegression` to plot the functions in the GAM that corresponds to the LogRR model (more generally, `visualize()` plots the GAM part of a LogRR model, excluding higher-degree rules). The plots show the sizes and shapes of the model's dependences on individual features. These can then be compared to a lending expert's knowledge. In the present case, all plots indicate that the model behaves as we would expect with some interesting nuances. 

The 36 features shown above involve only 14 of the original features in the data (not including the intercept), as verified below. For example, ExternalRiskEstimate appears in its unbinarized form in row 2 above and also in 3 rules (rows 14, 23, 34).

In [ ]:
dfx = lrr.explain()
# Separate 1st-degree rules into (feature, operation, value) to count unique features
dfx2 = dfx['rule/numerical feature'].str.split(' ', expand=True)
dfx2.columns = ['feature','operation','value']
dfx2['feature'].nunique() # includes intercept

It follows that there are 14 functions to plot, which we organize into semantic groups below to ease interpretation.

#### ExternalRiskEstimate
As expected from the BRCG Boolean rule above, 'ExternalRiskEstimate' is an important feature positively correlated with good credit risk. The jumps in the plot indicate that applicants with above average 'ExternalRiskEstimate' (the mean is 72) get an additional boost.

In [ ]:
lrr.visualize(data, fb, ['ExternalRiskEstimate']);

#### Credit inquiries
The next two plots illustrate the dependence on the applicant's credit inquiries. The first plot shows a significant penalty for having less than one month since the most recent inquiry ('MSinceMostRecentInqexcl7days' = 0).

In [ ]:
lrr.visualize(data, fb, ['MSinceMostRecentInqexcl7days']);

The second shows that predicted risk increases with the number of inquiries in the last six months ('NumInqLast6M').

In [ ]:
lrr.visualize(data, fb, ['NumInqLast6M']);

#### Debt level
The following four plots relate to the applicant's debt level. 'NetFractionRevolvingBurden' is the ratio of revolving debt (e.g. credit card) balance to credit limit, expressed as a percentage, and has a large negative impact on the probability of good credit. A small fraction of applicants (less than 1%) actually have NetFractionRevolvingBurden greater than 100%, i.e. more revolving debt than their credit limit. This might be investigated further by the data scientist.

In [ ]:
lrr.visualize(data, fb, ['NetFractionRevolvingBurden']);

The second 'NumBank2NatlTradesWHighUtilization' plot shows that the number of accounts ("trades") with high utilization (high balance relative to credit limit for each account) also has a large impact, with a drop as soon as one account has high utilization.

In [ ]:
lrr.visualize(data, fb, ['NumBank2NatlTradesWHighUtilization']);

<a name="prototypes"></a>
## 3. Loan Officer: Prototypical explanations for HELOC use case

We now show how to generate explanations in the form of selecting prototypical or similar user profiles to an applicant in question that a bank employee such as a loan officer may be interested in. This may help the employee understand the decision of an applicant's HELOC application being accepted or rejected in the context of other similar applications. Note that the selected prototypical applications are profiles that are part of the training set that has been used to train an AI model that predicts good or bad i.e. approved or rejected for these applications. In fact, the method used in this notebook can work even if we are given not just one but a set of user profiles for which we want to find similar profiles from a training dataset. Additionally, the method computes weights for each prototype showcasing its similarity to the user(s) in question.

The prototypical explanations in AIX360 are obtained using the Protodash algorithm developed in the following work: [ProtoDash: Fast Interpretable Prototype Selection](https://arxiv.org/abs/1707.01212)

We now provide a brief overview of the method. The method takes as input a datapoint (or group of datapoints) that we want to explain with respect to instances in a training set belonging to the same feature space. The method then tries to minimize the maximum mean discrepancy (MMD metric) between the datapoints we want to explain and a prespecified number of instances from the training set that it will select. In other words, it will try to select training instances that have the same distribution as the datapoints we want to explain. The method does greedy selection and has quality guarantees with it also returning importance weights for the chosen prototypical training instances indicative of how similar/representative they are.

In this tutorial, we will see two examples of obtaining prototypes, one for a user whose HELOC application was approved and another for a user whose HELOC application was rejected. In each case, we showcase the top five prototypes from the training data along with how similar the feature values were for these prototypes.

[Example 1. Obtaining similar samples as explanations for a HELOC applicant predicted as "Good"](#good)<br>
[Example 2. Obtaining similar samples as explanations for a HELOC applicant predicted as "Bad"](#bad)<br>


###### Why Protodash?
Before we showcase the two examples we provide some motivation for using this method. The method selects applications from the training set that are similar in different ways to the user application we want to explain. For example, a users loan may be rejected justifiably because the number of satisfactory trades he performed were low similar to another rejected user, or because his/her debts were too high similar to a different rejected user. Either of these reasons in isolation may be sufficient for rejection and the method is able to surface a variety of such reasons through the selected prototypes. This is not the case using standard nearest neighbor techniques which use metrics such as euclidean distance, cosine similarity amongst others, where one might get the same type of explanation (i.e. applications with only low number of satisfactory trades). Protodash thus is able to provide a much more well rounded and comprehensive view of why the decision for the applicant may be justifiable.

Another benefit of the method is that — since it does distribution matching between the user/users in question and those available in the training set — it could, in principle, be applied also in non-iid settings such as for time series data. Other approaches which find similar profiles using standard distance measures (viz. euclidean, cosine) do not have this property. Additionally, we can also highlight important features for the different prototypes that made them similar to the user/users in question.


In [ ]:
import tensorflow as tf
from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import Dense
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

from aix360.algorithms.contrastive import CEMExplainer, KerasClassifier
from aix360.algorithms.protodash import ProtodashExplainer
from aix360.datasets.heloc_dataset import HELOCDataset

### Load HELOC dataset and show sample applicants

In [ ]:
heloc = HELOCDataset()
df = heloc.dataframe()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 24)
pd.set_option('display.width', 1000)
print("Size of HELOC dataset:", df.shape)
print("Number of \"Good\" applicants:", np.sum(df['RiskPerformance']=='Good'))
print("Number of \"Bad\" applicants:", np.sum(df['RiskPerformance']=='Bad'))
print("Sample Applicants:")
df.head(10).transpose()

In [ ]:
# Plot (example) distributions for two features
print("Distribution of ExternalRiskEstimate and NumSatisfactoryTrades columns:")
hist = df.hist(column=['ExternalRiskEstimate', 'NumSatisfactoryTrades'], bins=10)

<a name="c1"></a>
### Step 1: Process and Normalize HELOC dataset for training

We will first process the HELOC dataset before using it to train an NN model that can predict the
target variable RiskPerformance. The HELOC dataset is a tabular dataset with numerical values. However, some of the values are negative and need to be filtered. The processed data is stored in the file heloc.npz for easy access. The dataset is also normalized for training.

The data processing and the type of model built in this case is different from the Data Scientist persona described above where rule based methods are showcased. This is the reason for going through these steps again for the Loan Officer persona.

#### a. Process the dataset

In [ ]:
# Clean data and split dataset into train/test
(Data, x_train, x_test, y_train_b, y_test_b) = heloc.split()

#### b. Normalize the dataset

In [ ]:
Z = np.vstack((x_train, x_test))
Zmax = np.max(Z, axis=0)
Zmin = np.min(Z, axis=0)

#normalize an array of samples to range [-0.5, 0.5]
def normalize(V):
    VN = (V - Zmin)/(Zmax - Zmin)
    VN = VN - 0.5
    return(VN)
    
# rescale a sample to recover original values for normalized values. 
def rescale(X):
    return(np.multiply ( X + 0.5, (Zmax - Zmin) ) + Zmin)

N = normalize(Z)
xn_train = N[0:x_train.shape[0], :]
xn_test  = N[x_train.shape[0]:, :]

<a name="c2"></a>
### Step 2. Define and train a NN classifier

Let us now build a loan approval model based on the HELOC dataset.

#### a. Define NN architecture
We now define the architecture of a 2-layer neural network classifier whose predictions we will try to interpret. 

In [ ]:
# nn with no softmax
def nn_small():
    model = Sequential()
    model.add(Dense(10, input_dim=23, kernel_initializer='normal', activation='relu'))
    model.add(Dense(2, kernel_initializer='normal'))    
    return model    

#### b. Train the NN

In [ ]:
# Set random seeds for repeatability
np.random.seed(1) 
tf.set_random_seed(2) 

class_names = ['Bad', 'Good']

# loss function
def fn(correct, predicted):
    return tf.nn.softmax_cross_entropy_with_logits(labels=correct, logits=predicted)

# compile and print model summary
nn = nn_small()
nn.compile(loss=fn, optimizer='adam', metrics=['accuracy'])
nn.summary()


# train model or load a trained model
TRAIN_MODEL = False

if (TRAIN_MODEL): 
    nn.fit(xn_train, y_train_b, batch_size=128, epochs=500, verbose=1, shuffle=False)
    nn.save_weights("heloc_nnsmall.h5")     
else:    
    nn.load_weights("heloc_nnsmall.h5")
        

# evaluate model accuracy        
score = nn.evaluate(xn_train, y_train_b, verbose=0) #Compute training set accuracy
#print('Train loss:', score[0])
print('Train accuracy:', score[1])

score = nn.evaluate(xn_test, y_test_b, verbose=0) #Compute test set accuracy
#print('Test loss:', score[0])
print('Test accuracy:', score[1])

<a name="good"></a>
### Step 3: Obtain similar samples as explanations for a HELOC applicant predicted as "Good" (Example 1)

#### a. Normalize the data and chose a particular applicant, whose profile is displayed below.

In [ ]:
p_train = nn.predict_classes(xn_train) # Use trained neural network to predict train points
p_train = p_train.reshape((p_train.shape[0],1))

z_train = np.hstack((xn_train, p_train)) # Store (normalized) instances that were predicted as Good
z_train_good = z_train[z_train[:,-1]==1, :]

zun_train = np.hstack((x_train, p_train)) # Store (unnormalized) instances that were predicted as Good 
zun_train_good = zun_train[zun_train[:,-1]==1, :]

Let us now consider applicant 8 whose loan was approved. Note that this applicant was also considered for the contrastive explainer, however, we now justify the approved status in a different manner using prototypical examples, which is arguably a better explanation for a bank employee.

In [ ]:
idx = 8

X = xn_test[idx].reshape((1,) + xn_test[idx].shape)

print("Chosen Sample:", idx)
print("Prediction made by the model:", class_names[np.argmax(nn.predict_proba(X))])
print("Prediction probabilities:", nn.predict_proba(X))
print("")

# attach the prediction made by the model to X
X = np.hstack((X, nn.predict_classes(X).reshape((1,1))))

Xun = x_test[idx].reshape((1,) + x_test[idx].shape) 
dfx = pd.DataFrame.from_records(Xun.astype('double')) # Create dataframe with original feature values
dfx[23] = class_names[int(X[0, -1])]
dfx.columns = df.columns
dfx.transpose()

#### b. Find similar applicants predicted as "good" using the protodash explainer. 

In [ ]:
explainer = ProtodashExplainer()
(W, S, setValues) = explainer.explain(X, z_train_good, m=5) # Return weights W, Prototypes S and objective function values

#### c. Display similar applicant user profiles and the extent to which they are similar to the chosen applicant as indicated by the last row in the table below labelled as "Weight".

In [ ]:
dfs = pd.DataFrame.from_records(zun_train_good[S, 0:-1].astype('double'))
RP=[]
for i in range(S.shape[0]):
    RP.append(class_names[int(z_train_good[S[i], -1])]) # Append class names
dfs[23] = RP
dfs.columns = df.columns  
dfs["Weight"] = np.around(W, 5)/np.sum(np.around(W, 5)) # Calculate normalized importance weights
dfs.transpose()

#### d. Compute how similar a feature of a prototypical user is to the chosen applicant.
The more similar the feature of prototypical user is to the applicant, the closer its weight is to 1. We can see below that several features for prototypes are quite similar to the chosen applicant. A human friendly explanation is provided thereafter.

In [ ]:
z = z_train_good[S, 0:-1] # Store chosen prototypes
eps = 1e-10 # Small constant defined to eliminate divide-by-zero errors
fwt = np.zeros(z.shape)
for i in range (z.shape[0]):
    for j in range(z.shape[1]):
        fwt[i, j] = np.exp(-1 * abs(X[0, j] - z[i,j])/(np.std(z[:, j])+eps)) # Compute feature similarity in [0,1]
                
# move wts to a dataframe to display
dfw = pd.DataFrame.from_records(np.around(fwt.astype('double'), 2))
dfw.columns = df.columns[:-1]
dfw.transpose()   

#### Explanation:
The above table depicts the five closest user profiles to the chosen applicant. Based on importance weight outputted by the method, we see that the prototype under column zero is the most representative user profile by far. This is (intuitively) confirmed from the feature similarity table above where more than 50% of the features (12 out of 23) of this prototype are identical to that of the chosen user whose prediction we want to explain. Also, the bank employee looking at the prototypical users and their features surmises that the approved applicant belongs to a group of approved users that have practically no debt (NetFractionInstallBurden). This justification gives the employee more confidence in approving the users application.

<a name="bad"></a>
### Example 2. Obtaining similar samples as explanations for a HELOC applicant predicted as "Bad". 
We now consider a user 1272 whose loan was denied. We obtained a contrastive explanation for this user before. Similar to user 8, we now obtain exemplar based explanations for this user to help the bank employee understand the reasons for the rejection. Steps similar to example 1 are followed in this case too, where we first process the data, obtain prototypes and their importance weights, and finally showcase how similar the features are of these prototypes to the user we want to explain.

#### a. Normalize the data and chose a particular applicant, whose profile is displayed below.

In [ ]:
z_train_bad = z_train[z_train[:,-1]==0, :]
zun_train_bad = zun_train[zun_train[:,-1]==0, :]

In [ ]:
idx = 1272 #another user to try 2385

X = xn_test[idx].reshape((1,) + xn_test[idx].shape)
print("Chosen Sample:", idx)
print("Prediction made by the model:", class_names[np.argmax(nn.predict_proba(X))])
print("Prediction probabilities:", nn.predict_proba(X))
print("")

X = np.hstack((X, nn.predict_classes(X).reshape((1,1))))

# move samples to a dataframe to display
Xun = x_test[idx].reshape((1,) + x_test[idx].shape)
dfx = pd.DataFrame.from_records(Xun.astype('double'))
dfx[23] = class_names[int(X[0, -1])]
dfx.columns = df.columns
dfx.transpose()

#### b. Find similar applicants predicted as "bad" using the protodash explainer. 

In [ ]:
(W, S, setValues) = explainer.explain(X, z_train_bad, m=5) # Return weights W, Prototypes S and objective function values

#### c. Display similar applicant user profiles and the extent to which they are similar to the chosen applicant as indicated by the last row in the table below labelled as "Weight".

In [ ]:
# move samples to a dataframe to display
dfs = pd.DataFrame.from_records(zun_train_bad[S, 0:-1].astype('double'))
RP=[]
for i in range(S.shape[0]):
    RP.append(class_names[int(z_train_bad[S[i], -1])]) # Append class names
dfs[23] = RP
dfs.columns = df.columns  
dfs["Weight"] = np.around(W, 5)/np.sum(np.around(W, 5)) # Compute normalized importance weights for prototypes
dfs.transpose()

#### d. Compute how similar a feature of a prototypical user is to the chosen applicant.
The more similar the feature of prototypical user is to the applicant, the closer its weight is to 1. We can see below that several features for prototypes are quite similar to the chosen applicant. Following this table we provide human friendly explanation based on this table.

In [ ]:
z = z_train_bad[S, 0:-1] # Store the prototypes
eps = 1e-10 # Small constant to guard against divide by zero errors
fwt = np.zeros(z.shape)
for i in range (z.shape[0]): # Compute feature similarity for each prototype
    for j in range(z.shape[1]):
        fwt[i, j] = np.exp(-1 * abs(X[0, j] - z[i,j])/(np.std(z[:, j])+eps))
                
# move wts to a dataframe to display
dfw = pd.DataFrame.from_records(np.around(fwt.astype('double'), 2))
dfw.columns = df.columns[:-1]
dfw.transpose()  

#### Explanation:
Here again, the above table depicts the five closest user profiles to the chosen applicant. Based on importance weight outputted by the method we see that the prototype under column zero is the most representative user profile by far. This is (intuitively) confirmed from the feature similarity table above where 10 features out of 23 of this prototype are highly similar (>0.9) to that of the user we want to explain. Also the bank employee can see that the applicant belongs to a group of rejected applicants with similar deliquency behavior. Realizing that the user also poses similar risk as these other applicants whose loan was rejected, the employee takes the more conservative decision of rejecting the users application as well.

<a name="contrastive"></a>
## 4. Customer: Contrastive explanations for HELOC Use Case

We now demonstrate how to compute contrastive explanations using AIX360 and how such explanations can help home owners understand the decisions made by AI models that approve or reject their HELOC applications. 

Typically, home owners would like to understand why they do not qualify for a line of credit and if so what changes in their application would qualify them. On the other hand, if they qualified, they might want to know what factors led to the approval of their application. 

In this context, contrastive explanations provide information to applicants about what minimal changes to their profile would have changed the decision of the AI model from reject to accept or vice-versa (_pertinent negatives_). For example, increasing the number of satisfactory trades to a certain value may have led to the acceptance of the application everything else being the same. 

The method presented here also highlights a minimal set of features and their values that would still maintain the original decision (_pertinent positives_). For example, for an applicant whose HELOC application was approved, the 
explanation may say that even if the number of satisfactory trades was reduced to a lower number, the loan would have still gotten through.

Additionally, organizations (Banks, financial institutions, etc.) would like to understand trends in the behavior of their AI models in approving loan applications, which could be done by studying contrastive explanations for individuals whose loans were either accepted or rejected. Looking at the aggregate statistics of pertinent positives for approved applicants the organization can get insight into what minimal set of features and their values play an important role in acceptances. While studying the aggregate statistics of pertinent negatives the organization can get insight into features that could change the status of rejected applicants and potentially uncover ways that an applicant may game the system by changing potentially non-important features that could alter the models outcome. 

The contrastive explanations in AIX360 are implemented using the algorithm developed in the following work:
###### [Explanations based on the Missing: Towards Contrastive Explanations with Pertinent Negatives](https://arxiv.org/abs/1802.07623)

We now provide a brief overview of the method. As mentioned above the algorithm outputs a contrastive explanation which consists of two parts: a) pertinent negatives (PNs) and b) pertinent positives (PPs). PNs identify a minimal set of features which if altered would change the classification of the original input. For example, in the loan case if a person's credit score is increased their loan application status may change from reject to accept. The manner in which the method accomplishes this is by optimizing a change in the prediction probability loss while enforcing an elastic norm constraint that results in minimal change of features and their values. Optionally, an auto-encoder may also be used to force these minimal changes to produce realistic PNs. PPs on the other hand identify a minimal set of features and their values that are sufficient to yield the original input's classification. For example, an individual's loan may still be accepted if the salary was 50K as opposed to 100K. Here again we have an elastic norm term so that the amount of information needed is minimal, however, the first loss term in this case tries to make the original input's class to be the winning class. For a more in-depth discussion, please refer to the above work.


The three main steps to obtain a contrastive explanation are shown below. The first two steps are more about processing the data and building an AI model while the third step computes the actual explanation. 

 [Step 1. Process and Normalize HELOC dataset for training](#c1)<br>
 [Step 2. Define and train a NN classifier](#c2)<br>
 [Step 3. Compute contrastive explanations for a few applicants](#c3)<br>


### Load HELOC dataset and show sample applicants

In [ ]:
import warnings
warnings.filterwarnings('ignore')

heloc = HELOCDataset()
df = heloc.dataframe()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 24)
pd.set_option('display.width', 1000)
print("Size of HELOC dataset:", df.shape)
print("Number of \"Good\" applicants:", np.sum(df['RiskPerformance']=='Good'))
print("Number of \"Bad\" applicants:", np.sum(df['RiskPerformance']=='Bad'))
print("Sample Applicants:")
df.head(10).transpose()

In [ ]:
# Plot (example) distributions for two features
print("Distribution of ExternalRiskEstimate and NumSatisfactoryTrades columns:")
hist = df.hist(column=['ExternalRiskEstimate', 'NumSatisfactoryTrades'], bins=10)

<a name="c1"></a>
### Step 1. Process and Normalize HELOC dataset for training

We will first process the HELOC dataset before using it to train an NN model that can predict the
target variable RiskPerformance. The HELOC dataset is a tabular dataset with numerical values. However, some of the values are negative and need to be filtered. The processed data is stored in the file heloc.npz for easy access. The dataset is also normalized for training.

The data processing and model building is very similar to the Loan Officer persona above, where ProtoDash was the method of choice. We repeat these steps here so that both the use cases can be run independently.

#### a. Process the dataset

In [ ]:
# Clean data and split dataset into train/test
PROCESS_DATA = False

if (PROCESS_DATA): 
    (Data, x_train, x_test, y_train_b, y_test_b) = heloc.split()
    np.savez('heloc.npz', Data=Data, x_train=x_train, x_test=x_test, y_train_b=y_train_b, y_test_b=y_test_b)
else:
    heloc = np.load('heloc.npz', allow_pickle = True)
    Data = heloc['Data']
    x_train = heloc['x_train']
    x_test  = heloc['x_test']
    y_train_b = heloc['y_train_b']
    y_test_b  = heloc['y_test_b']

#### b. Normalize the dataset

In [ ]:
Z = np.vstack((x_train, x_test))
Zmax = np.max(Z, axis=0)
Zmin = np.min(Z, axis=0)

#normalize an array of samples to range [-0.5, 0.5]
def normalize(V):
    VN = (V - Zmin)/(Zmax - Zmin)
    VN = VN - 0.5
    return(VN)
    
# rescale a sample to recover original values for normalized values. 
def rescale(X):
    return(np.multiply ( X + 0.5, (Zmax - Zmin) ) + Zmin)

N = normalize(Z)
xn_train = N[0:x_train.shape[0], :]
xn_test  = N[x_train.shape[0]:, :]

<a name="c2"></a>
### Step 2. Define and train a NN classifier

Let us now build a loan approval model based on the HELOC dataset.

#### a. Define NN architecture
We now define the architecture of a 2-layer neural network classifier whose predictions we will try to interpret. 

In [ ]:
# nn with no softmax
def nn_small():
    model = Sequential()
    model.add(Dense(10, input_dim=23, kernel_initializer='normal', activation='relu'))
    model.add(Dense(2, kernel_initializer='normal'))    
    return model  

#### b. Train the NN

In [ ]:
# Set random seeds for repeatability
np.random.seed(1) 
tf.set_random_seed(2) 

class_names = ['Bad', 'Good']

# loss function
def fn(correct, predicted):
    return tf.nn.softmax_cross_entropy_with_logits(labels=correct, logits=predicted)

# compile and print model summary
nn = nn_small()
nn.compile(loss=fn, optimizer='adam', metrics=['accuracy'])
nn.summary()


# train model or load a trained model
TRAIN_MODEL = True

if (TRAIN_MODEL):             
    nn.fit(xn_train, y_train_b, batch_size=128, epochs=500, verbose=1, shuffle=False)
    nn.save_weights("heloc_nnsmall.h5")     
else:    
    nn.load_weights("heloc_nnsmall.h5")
        

# evaluate model accuracy        
score = nn.evaluate(xn_train, y_train_b, verbose=0) #Compute training set accuracy
#print('Train loss:', score[0])
print('Train accuracy:', score[1])

score = nn.evaluate(xn_test, y_test_b, verbose=0) #Compute test set accuracy
#print('Test loss:', score[0])
print('Test accuracy:', score[1])

<a name="c3"></a>
### Step 3. Compute contrastive explanations for a few applicants

Given the trained NN model to decide on loan approvals, let us first examine an applicant whose application was denied and what (minimal) changes to his/her application would lead to approval (i.e. finding pertinent negatives). We will then look at another applicant whose loan was approved and ascertain features that would minimally suffice in him/her still getting a positive outcome (i.e. finding pertinent positives).

#### a. Compute Pertinent Negatives (PN): 

In order to compute pertinent negatives, the CEM explainer computes a user profile that is close to the original applicant but for whom the decision of HELOC application is different. The explainer alters a minimal set of features by a minimal (positive) amount. This will help the user whose loan application was initially rejected say, to ascertain how to get it accepted. 

In [ ]:
# Some interesting user samples to try: 2344 449 1168 1272
idx = 1272

X = xn_test[idx].reshape((1,) + xn_test[idx].shape)
print("Computing PN for Sample:", idx)
print("Prediction made by the model:", nn.predict_proba(X))
print("Prediction probabilities:", class_names[np.argmax(nn.predict_proba(X))])
print("")

mymodel = KerasClassifier(nn)
explainer = CEMExplainer(mymodel)

arg_mode = 'PN' # Find pertinent negatives
arg_max_iter = 1000 # Maximum number of iterations to search for the optimal PN for given parameter settings
arg_init_const = 10.0 # Initial coefficient value for main loss term that encourages class change
arg_b = 9 # No. of updates to the coefficient of the main loss term
arg_kappa = 0.1 # Minimum confidence gap between the PNs (changed) class probability and original class' probability
arg_beta = 1e-1 # Controls sparsity of the solution (L1 loss)
arg_gamma = 100 # Controls how much to adhere to a (optionally trained) auto-encoder
my_AE_model = None # Pointer to an auto-encoder

# Find PN for applicant 1272
(adv_pn, delta_pn, info_pn) = explainer.explain_instance(X, arg_mode, my_AE_model, arg_kappa, arg_b,
                                                         arg_max_iter, arg_init_const, arg_beta, arg_gamma)

Let us start by examining one particular loan application that was denied for applicant 1272. We showcase below how the decision could have been different through minimal changes to the profile conveyed by the pertinent negative. We also indicate the importance of different features to produce the change in the application status. The column delta in the table below indicates the necessary deviations for each of the features to produce this change. A human friendly explanation is then provided based on these deviations following the feature importance plot.

In [ ]:
Xpn = adv_pn
classes = [ class_names[np.argmax(nn.predict_proba(X))], class_names[np.argmax(nn.predict_proba(Xpn))], 'NIL' ]

print("Sample:", idx)
print("prediction(X)", nn.predict_proba(X), class_names[np.argmax(nn.predict_proba(X))])
print("prediction(Xpn)", nn.predict_proba(Xpn), class_names[np.argmax(nn.predict_proba(Xpn))] )


X_re = rescale(X) # Convert values back to original scale from normalized
Xpn_re = rescale(Xpn)
Xpn_re = np.around(Xpn_re.astype(np.double), 2)

delta_re = Xpn_re - X_re
delta_re = np.around(delta_re.astype(np.double), 2)
delta_re[np.absolute(delta_re) < 1e-4] = 0

X3 = np.vstack((X_re, Xpn_re, delta_re))

dfre = pd.DataFrame.from_records(X3) # Create dataframe to display original point, PN and difference (delta)
dfre[23] = classes

dfre.columns = df.columns
dfre.rename(index={0:'X',1:'X_PN', 2:'(X_PN - X)'}, inplace=True)
dfret = dfre.transpose()


def highlight_ce(s, col, ncols):
    if (type(s[col]) != str):
        if (s[col] > 0):
            return(['background-color: yellow']*ncols)    
    return(['background-color: white']*ncols)

dfret.style.apply(highlight_ce, col='(X_PN - X)', ncols=3, axis=1) 

Now let us compute the importance of different PN features that would be instrumental in 1272 receiving a favorable outcome and display below.

In [ ]:
plt.rcdefaults()
fi = abs((X-Xpn).astype('double'))/np.std(xn_train.astype('double'), axis=0) # Compute PN feature importance
objects = df.columns[-2::-1]
y_pos = np.arange(len(objects))
performance = fi[0, -1::-1]

plt.barh(y_pos, performance, align='center', alpha=0.5) # bar chart
plt.yticks(y_pos, objects) # Display features on y-axis
plt.xlabel('weight') # x-label
plt.title('PN (feature importance)') # Heading

plt.show() # Display PN feature importance

#### Explanation:  
We observe that the applicant 1272's loan application would have been accepted if the consolidated risk marker score  (i.e. ExternalRiskEstimate) increased from 65 to 76, the loan application was on file (i.e. AverageMlnFile) for about 65 months and if the number of satisfactory trades (i.e. NumSatisfactoryTrades) increased to little over 20.

_The above changes to the three suggested factors are also intuitively consistent in improving the chances of acceptance of an application, since all three are monotonic with probability of acceptance (refer HELOC description table). 
However, one must realize that the above explanation is for the particular applicant based on what the model would do and does not necessarily have to agree with their intuitive meaning. In fact, if the explanation is deemed unacceptable then its an indication that perhaps the model should be debugged/updated_.

#### Compute Pertinent Positives (PP):
In order to compute pertinent positives, the CEM explainer identifies a minimal set of features along with their values (as close to 0) that would still maintain the predicted loan application status of the applicant.

In [ ]:
# Some interesting user samples to try: 8 9 11
idx = 8

X = xn_test[idx].reshape((1,) + xn_test[idx].shape)
print("Computing PP for Sample:", idx)
print("Prediction made by the model:", class_names[np.argmax(nn.predict_proba(X))])
print("Prediction probabilities:", nn.predict_proba(X))
print("")


mymodel = KerasClassifier(nn)
explainer = CEMExplainer(mymodel)

arg_mode = 'PP' # Find pertinent positives
arg_max_iter = 1000 # Maximum number of iterations to search for the optimal PN for given parameter settings
arg_init_const = 10.0 # Initial coefficient value for main loss term that encourages class change
arg_b = 9 # No. of updates to the coefficient of the main loss term
arg_kappa = 0.1 # Minimum confidence gap between the PNs (changed) class probability and original class' probability
arg_beta = 1e-1 # Controls sparsity of the solution (L1 loss)
arg_gamma = 100 # Controls how much to adhere to a (optionally trained) auto-encoder
my_AE_model = None # Pointer to an auto-encoder

(adv_pp, delta_pp, info_pp) = explainer.explain_instance(X, arg_mode, my_AE_model, arg_kappa, arg_b,
                                                         arg_max_iter, arg_init_const, arg_beta, arg_gamma)

For the pertinent positives, we look at a different applicant 8 whose loan application was approved. We want to ascertain here what minimal values for this profile would still have lead to acceptance. Below, we showcase the pertinent positive as well as the important features in maintaining the approved status. The 0s in the PP column indicate that those features were not important. The 0s in the PP column indicate that those features were not important. Here too, we provide a human friendly explanation following the feature importance plot.

In [ ]:
Xpp = delta_pp
classes = [ class_names[np.argmax(nn.predict_proba(X))], class_names[np.argmax(nn.predict_proba(Xpp))]]

print("PP for Sample:", idx)
print("Prediction(Xpp) :", class_names[np.argmax(nn.predict_proba(Xpp))])
print("Prediction probabilities for Xpp:", nn.predict_proba(Xpp))
print("")

X_re = rescale(X) # Convert values back to original scale from normalized
adv_pp_re = rescale(adv_pp)
Xpp_re = X_re - adv_pp_re
Xpp_re = np.around(Xpp_re.astype(np.double), 2)
Xpp_re[Xpp_re < 1e-4] = 0

X2 = np.vstack((X_re, Xpp_re))

dfpp = pd.DataFrame.from_records(X2.astype('double')) # Showcase a dataframe for the original point and PP
dfpp[23] = classes
dfpp.columns = df.columns
dfpp.rename(index={0:'X',1:'X_PP'}, inplace=True)
dfppt = dfpp.transpose()

dfppt.style.apply(highlight_ce, col='X_PP', ncols=2, axis=1) 

In [ ]:
plt.rcdefaults()
fi = abs(Xpp_re.astype('double'))/np.std(x_train.astype('double'), axis=0) # Compute PP feature importance
    
objects = df.columns[-2::-1]
y_pos = np.arange(len(objects)) # Get input feature names
performance = fi[0, -1::-1]

plt.barh(y_pos, performance, align='center', alpha=0.5) # Bar chart
plt.yticks(y_pos, objects) # Plot feature names on y-axis
plt.xlabel('weight') #x-label
plt.title('PP (feature importance)') # Figure heading

plt.show()    # Display the feature importance

#### Explanation:  
We observe that the applicant 8's loan application would still have been accepted even if the consolidated risk marker score (i.e. ExternalRiskEstimate) reduced from 82 to around 40, application was on file (i.e. AverageMlnFile) for close to 70 months and number of satisfactory trades (i.e. NumSatisfactoryTrades) reduced from 22 to almost single digits.

_Note that explanations may change a bit based on equivalent values in a local minima._